Binding

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document

In [2]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

C:\Users\msshe\AppData\Local\Temp\ipykernel_25904\2127729888.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
c:\Users\msshe\Documents\Projects\Demo-Langchain\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
texts = [
    "Bangalore is a major tech hub in India.",
    "Paris is famous for the Eiffel Tower.",
    "The Amazon rainforest is vital for the planet."
]

In [4]:
docs = [Document(page_content=txt) for txt in texts]

In [5]:
vectorstore = FAISS.from_documents(docs, embedding_model)

In [6]:
query = "Which city has the Eiffel Tower?"
result = vectorstore.similarity_search(query, k=1)

In [7]:
print("Query result:", result[0].page_content)

Query result: Paris is famous for the Eiffel Tower.


bind()

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage, SystemMessage

llm_model=ChatGroq(groq_api_key = os.getenv("GROQ_API_KEY"),
                   model_name = "llama3-70b-8192")

In [12]:
prompt = ChatPromptTemplate.from_template(
    "Explain the concept of {concept}."
)

In [13]:
parser = StrOutputParser()

In [14]:
chain = prompt | llm_model.bind(stop = "SOLUTION") | parser

In [16]:
response = chain.invoke({"concept":"machine learning"})

In [17]:
print(response)

Machine learning is a subfield of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance over time.

The core concept of machine learning is to enable machines to automatically learn from experience, adapt to new data, and make predictions or decisions based on that data. This is achieved by training machine learning models on large datasets, which allows them to identify patterns, relationships, and trends within the data.

Machine learning involves a range of techniques, including:

1. **Supervised learning**: The model is trained on labeled data, where the correct output is already known. The model learns to map inputs to outputs based on the labeled data**.
2. **Unsupervised learning**: The model is trained on unlabeled data, and it must find patterns or relationships in the data on its own.
3. **Reinforcement learning**: The model learns by interacting with an envi

Combining LCEL chains

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()


In [20]:
fact_prompt = ChatPromptTemplate.from_template(
    "Tell me a curious fact about {politician}"
)
fact_chain = fact_prompt | llm_model | parser

In [21]:
judgement_prompt = ChatPromptTemplate.from_template(
    "Based on the following fact, was the politician good for humanity? Answer yes or no and briefly explain.\n\nFact: {fact}"
)
judgement_chain = {"fact": fact_chain} | judgement_prompt | llm_model | parser


In [22]:
final_chain = judgement_chain

# Example run
output = final_chain.invoke({"politician": "Narendra Modi"})
print(output)

Yes.

The fact that Narendra Modi, as Prime Minister of India, has promoted conservation efforts for the Asiatic Lion habitat in Gir National Park, contributing to the species' preservation, is a positive action that benefits humanity and the environment.


itemgetter

In [ ]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

from langchain_core.output_parsers import StrOutputParser


parser = StrOutputParser()


In [24]:
prompt_1 = ChatPromptTemplate.from_template("Tell me a curious fact about {politician}")

chain_1 = prompt_1 | llm_model| parser


In [25]:
# This lets us fetch only 'politician' from the original input dictionary
get_politician = RunnableLambda(itemgetter("politician"))


In [27]:
prompt_2 = ChatPromptTemplate.from_template("Was {politician} good for humanity?")

chain_2 = get_politician | prompt_2 | llm_model | parser


In [30]:

from langchain_core.runnables import RunnableParallel

# Run both chains in parallel
composed_chain = RunnableParallel({
    "fact": chain_1,
    "verdict": chain_2
})


In [32]:
result = composed_chain.invoke({"politician": "Abraham Lincoln"})
print(result['verdict'])


What a timely and thought-provoking question. Abraham Lincoln, the President of the United States from 1861 until his assassination in 1865, is widely regarded as one of the most influential figures in American history. His legacy has far-reaching implications for humanity, with both positive and negative perspectives. Here's a balanced analysis:

**Positive impacts on humanity:**

1. **End of Slavery**: He issued the Emancator Proclamation (1863), which declared freedom for all slaves in Confederate territories, paving the way for the eventual abolition of slavery with the 13th Amendment to the Constitution (1865). This monumental step towards equality and human rights has inspired struggles for freedom and civil rights worldwide.
2. **Preservation of the Union**: Lincoln's leadership during the American Civil War (1861-1865) ensured the unity of the United States, which has since become a beacon of stability and democracy, influencing the development of modern nation-states and inter

In [5]:
prompt1 = ChatPromptTemplate.from_template("what is the country {politician} from ?")
prompt2 = ChatPromptTemplate.from_template("what continent is the {country} in? respond in {language} ")

In [6]:
chain1 = prompt1 | llm_model | StrOutputParser()
chain2 = (
    {"country": chain1, "language": RunnableLambda(itemgetter("language"))}
    | prompt2
    | llm_model
    | StrOutputParser()
    
)

In [45]:
chain2.invoke({
    "politician" : "Modi",
    "language" : "hindi"
})

"You're referring to Narendra Damodardास मोदी!\n\nवह भारत के वर्तमान प्रधानमंत्री हैं। उनका जन्म 17 सितंबर 1950 को गुजरात के वडनगर में हुआ था। वे 2001 से 2014 तक गुजरात के मुख्यमंत्री रहे और 2014 से भारत के प्रधानमंत्री हैं, भारतीय संसद में भारतीय जनता पार्टी (भाजपा) का नेतृत्व कर रहे हैं।\n\ncontinent: एशिया (Asia)"